<a href="https://colab.research.google.com/github/naoya1110/IP2_Lecture_2023/blob/main/2023_IP2_Step10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step10 NumpyとOpenCVを使った画像処理

## はじめに
NumpyとOpenCVを使って簡単な画像処理をしてみましょう。

## パッケージのインポート
Numpy，Matplotlibに加えて画像処理用のパッケージOpenCV（インポート名は`cv2`）を使用します。



```
import numpy as np                  # Numpyのインポート
import matplotlib.pyplot as plt     # Matplotlibのインポート
import cv2                          # OpenCVのインポート
```



## Google Driveのマウント


```
from google.colab import drive
drive.mount('/content/drive')
```



## 画像データの読み込みと表示
`cat.jpg`を読み込んでみましょう。



```
filepath = "PATH/TO/YOUR/DATA"    # 要変更
img = cv2.imread(filepath)
```



`img`のデータタイプを確認してみましょう。Numpy配列になっていることが分かります。



```
type(img)
```



`img`に含まれる要素の最大値と最小値を確認しましょう。



```
print("最大値:", img.max())
print("最小値:", img.min())
```



`img`の要素のデータタイプを確認しましょう。`uint8`（符号なし8ビット整数）になっていることが分かります。`uint8`は2進数で$(0000\,0000)_2$から$(1111\,1111)_2$まで，10進数では0から255までを表すことができます。



```
img.dtype
```



imgのshapeを確認してみましょう。



```
img.shape
```



487x650x3の3次元配列になっていることが分かります。これは縦487ピクセル，横650ピクセルの画像であることを示しています。最後の3はカラーチャンネルと呼ばれるものです。カラー画像は赤(Red), 緑(Green), 青(Blue)の光の3原色(RGB)の2次元配列を重ね合わせることで構成されています。

`matplotlib`を使って`img`を表示してみましょう。



```
plt.imshow(img)    # 画像データの表示
```



なぜか画像が青っぽくなってしまいました。これは画像の読み込みに利用したOpenCVと画像の表示に利用したMatplotlibでカラーチャンネルの順番が異なるためです。OpenCVはB --> G --> R，MatplotlibはR --> B --> Gとして扱います。従って以下のようにカラーチャンネルの順番を変更します。



```
img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # 色変換 BGR ---> RGB
plt.imshow(img_RGB)
```



## 画像データの色
R, G, Bの各チャンネルのデータのみをスライスするには次のようにします。



```
img_R = img_RGB[:, :, 0]    # Rチャンネルのみ
img_G = img_RGB[:, :, 1]    # Gチャンネルのみ
img_B = img_RGB[:, :, 2]    # Bチャンネルのみ
```



Rチャンネルのデータのみを表示してみましょう。



```
plt.imshow(img_R, cmap="Reds")
plt.colorbar()
```



Gチャンネルのデータのみを表示してみましょう。



```
plt.imshow(img_G, cmap="Greens")
plt.colorbar()
```



Bチャンネルのデータのみを表示してみましょう。



```
plt.imshow(img_G, cmap="Blues")
plt.colorbar()
```



ここで猫のお尻のあたりのデータに注目してみましょう。カラー画像では白くなっている部分が，R, G, Bの各チャンネルのデータで濃い色（255に近い値）となっていることが分かります。一方，ベンチの脚など，カラー画像では黒くなっている部分では，R, G, Bの各チャンネルのデータが白い色（0に近い値）になっていることが分かります。つまりカラー画像を作るときは，R, G, Bの各値を最大値の255に近づけると白くなり，逆に0に近づけると黒い色になります。これを「加法混色」と言います。

参考：https://www.dataplan.jp/blog/design/456

次のセルでred_value, green_value, blue_valueの値を変更して合成される色がどのように変化するか確認しましょう。

参考　https://www.sysecol2.ethz.ch/_DevAux/Testing_and_Development/colormap.html



```
red_value = 255
green_value = 120
blue_value = 75

test_img = np.zeros((5, 5, 3), dtype="uint8")   # 5x5x3の配列，要素の値は全て0で初期化

test_img[:, :, 0] = red_value        # redチャンネルの要素を全てred_valueにする
test_img[:, :, 1] = green_value      # greenチャンネルの要素を全てgreen_valueにする
test_img[:, :, 2] = blue_value       # blueチャンネルの要素を全てblue_valueにする

plt.imshow(test_img)
```



## 色調整

### ガンマ補正
画像データの明るさを調整する方法としてガンマ補正と呼ばれる手法があります。ガンマ補正は元画像$\mathrm{img}_\mathrm{before}$の各ピクセルの値に対してガンマ$\gamma$の値を用いて次の式を計算し，補正後の画像$\mathrm{img}_\mathrm{after}$を生成します。

$\displaystyle \mathrm{img}_\mathrm{after} = 255 \times \left( \dfrac{\mathrm{img}_\mathrm{before}}{{255}}\right)^\gamma$

参考：https://qiita.com/yoya/items/122b93970c190068c752

元画像の各ピクセルの値`x`と補正後の画像の各ピクセルの値`y`は次の関係になります。



```
plt.figure(figsize=(5, 5))
x = np.arange(0, 256, 1)

for gamma in [0.25, 0.5, 1, 2, 4]:
    y = 255*(x/255)**gamma
    plt.plot(x, y, label=f"gamma={gamma:.2f}")

plt.xlabel("Input, x")
plt.ylabel("Output, y")
plt.grid()
plt.legend()
```



実際にガンマ補正を適用してみましょう。



```
gamma = 0.5

img_before = img_RGB.copy()                # img_RGBをimg_beforeにコピー
img_after = 255*(img_before/255)**gamma    # ガンマ補正，データタイプは自動的にfloatに変更される
img_after = img_after.astype("uint8")      # データをuint8に変更

plt.subplot(1, 2, 1)
plt.imshow(img_before)
plt.title("before")

plt.subplot(1, 2, 2)
plt.imshow(img_after)
plt.title("after")
```



補正の前後でRGBの各ピクセルの値がどのような分布をしているのかヒストグラムを確認してみましょう。



```
bins = np.arange(0, 256, 5)
colors = ["red", "green", "blue"]

plt.figure(figsize=(8, 4))
for i in range(3):
    plt.subplot(2, 3, i+1)
    plt.hist(img_before[:, :, i].reshape(-1), bins=bins, color=colors[i], label=colors[i], alpha=0.3)
    plt.legend()
    plt.title("Before")
    
    plt.subplot(2, 3, i+4)
    plt.hist(img_after[:, :, i].reshape(-1), bins=bins, color=colors[i], label=colors[i], alpha=0.3)
    plt.legend()
    plt.title("After")

plt.tight_layout()
```



### 画像の保存



```
savepath = "img_after.jpg"                               # 保存先のパス
img_after = cv2.cvtColor(img_after, cv2.COLOR_RGB2BGR)   # 色変換 RGB ---> BGR
cv2.imwrite(savepath, img_after)                         # savepathにimg_afterの画像データを保存
```



### グレースケールと二値化

カラー画像からグレイスケール画像へ変換することもできます。



```
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)   # 色変換 BGR ---> GRAY
plt.imshow(img_gray, cmap="gray")
```



img_grayのshapeを確認するとカラーチャンネルが無くなったため487x650の二次元配列になっていることが分かります。



```
img_gray.shape
```



グレースケール画像から二値化画像に変更します。二値化画像とは全てのピクセルを白（255）または黒（0）で表した画像のことです。元画像に対して閾値`threshold`を設定し二値化を行います。



```
threshold = 120         # 閾値を設定

img_binary = img_gray.copy()         # img_grayをコピーしてimg_binaryを作成

img_binary[img_gray >= threshold] = 255    # threshold以上の要素は全て255に変更
img_binary[img_gray < threshold] = 0       # threshold未満の要素は全て0に変更

plt.imshow(img_binary, cmap="gray")
```



### 画像の保存



```
savepath = "img_binary.jpg"
cv2.imwrite(savepath, img_binary)
```



## サイズ変更
画像のサイズを変更(resize)することができます。Numpyのreshapeと違って要素数に関係なくどんなサイズでも変更可能です。



```
new_size = (200, 50)                          # 新しい画像サイズ
img_resized = cv2.resize(img_RGB, new_size)   # サイズ変更
plt.imshow(img_resized)
```



In [ ]:
new_size = (200, 50)                         # 新しい画像サイズ
img_resized = cv2.resize(img_RGB, new_size)   # サイズ変更
plt.figure(figsize=(5, 2.5))
plt.imshow(img_resized)